# Mga Mekanismo ng Atensyon at mga Transformer

Isa sa mga pangunahing kahinaan ng mga recurrent network ay ang lahat ng salita sa isang sequence ay may parehong epekto sa resulta. Nagdudulot ito ng hindi optimal na performance sa mga karaniwang LSTM encoder-decoder na modelo para sa mga sequence-to-sequence na gawain, tulad ng Named Entity Recognition at Machine Translation. Sa totoong buhay, may mga partikular na salita sa input sequence na mas may epekto sa mga output kaysa sa iba.

Isaalang-alang ang isang sequence-to-sequence na modelo, tulad ng machine translation. Ito ay ipinatutupad gamit ang dalawang recurrent network, kung saan ang isang network (**encoder**) ay nagko-collapse ng input sequence sa isang hidden state, at ang isa pa, ang **decoder**, ay nag-u-unroll ng hidden state na ito sa isinaling resulta. Ang problema sa ganitong paraan ay ang huling estado ng network ay nahihirapang maalala ang simula ng pangungusap, na nagdudulot ng mababang kalidad ng modelo sa mahahabang pangungusap.

Ang **Mekanismo ng Atensyon** ay nagbibigay ng paraan upang timbangin ang kontekstwal na epekto ng bawat input vector sa bawat output prediction ng RNN. Ang paraan ng pagpapatupad nito ay sa pamamagitan ng paglikha ng mga shortcut sa pagitan ng mga intermediate state ng input RNN at output RNN. Sa ganitong paraan, kapag gumagawa ng output symbol $y_t$, isasaalang-alang natin ang lahat ng input hidden states $h_i$, na may iba't ibang timbang na coefficients $\alpha_{t,i}$.

![Larawan na nagpapakita ng encoder/decoder model na may additive attention layer](../../../../../translated_images/tl/encoder-decoder-attention.7a726296894fb567.webp)  
*Ang encoder-decoder model na may additive attention mechanism mula sa [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), na binanggit mula sa [blog post na ito](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Ang attention matrix $\{\alpha_{i,j}\}$ ay kumakatawan sa antas kung saan ang ilang input na salita ay may papel sa pagbuo ng isang partikular na salita sa output sequence. Narito ang halimbawa ng ganitong matrix:

![Larawan na nagpapakita ng sample alignment na natagpuan ng RNNsearch-50, kinuha mula sa Bahdanau - arviz.org](../../../../../translated_images/tl/bahdanau-fig3.09ba2d37f202a6af.webp)  

*Larawan mula sa [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Ang mga mekanismo ng atensyon ay responsable para sa karamihan ng kasalukuyan o malapit sa kasalukuyang state-of-the-art sa Natural Language Processing. Gayunpaman, ang pagdaragdag ng atensyon ay lubos na nagpapataas ng bilang ng mga parameter ng modelo na nagdulot ng mga isyu sa scaling sa mga RNN. Ang isang pangunahing limitasyon ng pag-scale ng mga RNN ay ang likas na katangian ng mga modelong ito na nagpapahirap sa pag-batch at pag-parallelize ng training. Sa isang RNN, ang bawat elemento ng isang sequence ay kailangang iproseso nang sunud-sunod, na nangangahulugang hindi ito madaling ma-parallelize.

Ang pagsasama ng mga mekanismo ng atensyon sa limitasyong ito ay humantong sa paglikha ng mga Transformer Models na ngayon ay itinuturing na State of the Art, tulad ng BERT at OpenGPT3.

## Mga Transformer Model

Sa halip na ipasa ang konteksto ng bawat nakaraang prediksyon sa susunod na hakbang ng pagsusuri, ang **transformer models** ay gumagamit ng **positional encodings** at atensyon upang makuha ang konteksto ng isang input sa loob ng isang ibinigay na window ng teksto. Ang larawan sa ibaba ay nagpapakita kung paano nakukuha ng positional encodings na may atensyon ang konteksto sa loob ng isang window.

![Animated GIF na nagpapakita kung paano isinasagawa ang mga pagsusuri sa transformer models.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)  

Dahil ang bawat input na posisyon ay independiyenteng naiuugnay sa bawat output na posisyon, ang mga transformer ay mas mahusay sa pag-parallelize kaysa sa mga RNN, na nagbibigay-daan sa mas malalaking at mas expressive na mga modelo ng wika. Ang bawat attention head ay maaaring gamitin upang matutunan ang iba't ibang relasyon sa pagitan ng mga salita na nagpapabuti sa mga gawain sa Natural Language Processing.

Ang **BERT** (Bidirectional Encoder Representations from Transformers) ay isang napakalaking multi-layer transformer network na may 12 layer para sa *BERT-base*, at 24 para sa *BERT-large*. Ang modelo ay unang pre-trained sa malaking corpus ng text data (WikiPedia + mga libro) gamit ang unsupervised training (pagpapredikta ng mga masked na salita sa isang pangungusap). Sa panahon ng pre-training, ang modelo ay sumisipsip ng makabuluhang antas ng pag-unawa sa wika na maaaring magamit sa iba pang mga dataset gamit ang fine tuning. Ang prosesong ito ay tinatawag na **transfer learning**.

![Larawan mula sa http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/tl/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)  

Maraming mga bersyon ng Transformer architectures kabilang ang BERT, DistilBERT, BigBird, OpenGPT3, at iba pa na maaaring i-fine tune. Ang [HuggingFace package](https://github.com/huggingface/) ay nagbibigay ng repository para sa pag-train ng marami sa mga arkitekturang ito gamit ang PyTorch.

## Paggamit ng BERT para sa Text Classification

Tingnan natin kung paano natin magagamit ang pre-trained na BERT model para sa paglutas ng ating tradisyunal na gawain: sequence classification. Ika-classify natin ang orihinal na AG News dataset.

Una, i-load natin ang HuggingFace library at ang ating dataset:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Dahil gagamit tayo ng pre-trained na BERT model, kailangan natin gumamit ng partikular na tokenizer. Una, iloload natin ang tokenizer na konektado sa pre-trained na BERT model.

Ang HuggingFace library ay may repository ng mga pre-trained na modelo, na maaari mong gamitin sa pamamagitan lamang ng pag-specify ng kanilang mga pangalan bilang argumento sa mga `from_pretrained` na function. Ang lahat ng kinakailangang binary files para sa modelo ay awtomatikong mada-download.

Gayunpaman, sa ilang pagkakataon, maaaring kailanganin mong i-load ang sarili mong mga modelo. Sa ganitong kaso, maaari mong i-specify ang directory na naglalaman ng lahat ng kaugnay na files, kabilang ang mga parameter para sa tokenizer, `config.json` file na may mga parameter ng modelo, binary weights, at iba pa.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Ang `tokenizer` na object ay naglalaman ng `encode` na function na maaaring direktang gamitin upang i-encode ang teksto:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Pagkatapos, gumawa tayo ng mga iterator na gagamitin natin sa panahon ng pagsasanay upang ma-access ang data. Dahil gumagamit ang BERT ng sarili nitong encoding function, kailangan nating magtakda ng isang padding function na katulad ng `padify` na naitakda natin dati:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

Sa ating kaso, gagamit tayo ng pre-trained BERT model na tinatawag na `bert-base-uncased`. I-load natin ang model gamit ang `BertForSequenceClassification` package. Tinitiyak nito na ang ating model ay mayroon nang kinakailangang arkitektura para sa classification, kabilang ang final classifier. Makakakita ka ng warning message na nagsasabing ang weights ng final classifier ay hindi pa initialized, at ang model ay mangangailangan ng pre-training - okay lang iyon, dahil eksakto iyon ang gagawin natin!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Ngayon, handa na tayong magsimula ng pagsasanay! Dahil ang BERT ay pre-trained na, nais nating magsimula sa medyo maliit na learning rate upang hindi masira ang mga initial weights.

Ang lahat ng mahahalagang gawain ay ginagawa ng modelong `BertForSequenceClassification`. Kapag tinawag natin ang modelo sa training data, ibinabalik nito ang parehong loss at network output para sa input minibatch. Ginagamit natin ang loss para sa parameter optimization (`loss.backward()` ang gumagawa ng backward pass), at ang `out` para sa pagkalkula ng training accuracy sa pamamagitan ng paghahambing ng nakuha na mga label `labs` (kinakalkula gamit ang `argmax`) sa inaasahang `labels`.

Upang makontrol ang proseso, iniipon natin ang loss at accuracy sa loob ng ilang mga iteration, at ipinapakita ang mga ito tuwing `report_freq` na training cycles.

Ang pagsasanay na ito ay malamang na tatagal ng medyo mahabang panahon, kaya nililimitahan natin ang bilang ng mga iteration.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Makikita mo (lalo na kung dadagdagan mo ang bilang ng mga iteration at maghintay nang mas matagal) na ang BERT classification ay nagbibigay sa atin ng medyo magandang accuracy! Ito ay dahil ang BERT ay may mahusay na pag-unawa sa istruktura ng wika, at kailangan lang nating i-fine-tune ang final classifier. Gayunpaman, dahil ang BERT ay isang malaking modelo, ang buong proseso ng training ay tumatagal ng mahabang oras at nangangailangan ng matinding computational power! (GPU, at mas mainam kung higit sa isa).

> **Note:** Sa ating halimbawa, gumagamit tayo ng isa sa pinakamaliit na pre-trained BERT models. May mas malalaking modelo na malamang magbibigay ng mas magagandang resulta.


## Pagsusuri sa pagganap ng modelo

Ngayon, maaari na nating suriin ang pagganap ng ating modelo sa test dataset. Ang evaluation loop ay halos katulad ng training loop, ngunit huwag kalimutang ilipat ang modelo sa evaluation mode sa pamamagitan ng pagtawag sa `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Mga Mahalagang Punto

Sa yunit na ito, nakita natin kung gaano kadali ang paggamit ng pre-trained na language model mula sa **transformers** library at iangkop ito sa ating gawain sa text classification. Katulad nito, maaaring gamitin ang mga BERT model para sa entity extraction, question answering, at iba pang mga gawain sa NLP.

Ang mga Transformer model ay kumakatawan sa kasalukuyang pinakamataas na antas ng teknolohiya sa NLP, at sa karamihan ng mga kaso, ito ang dapat na unang solusyon na subukan kapag gumagawa ng mga custom na solusyon sa NLP. Gayunpaman, ang pag-unawa sa mga pangunahing prinsipyo ng recurrent neural networks na tinalakay sa module na ito ay napakahalaga kung nais mong bumuo ng mga advanced na neural model.



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
